In [26]:
!pip install torch torchvision opencv-python dlib matplotlib numpy

In [27]:
import cv2
import dlib

# Load pre-trained face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
# Replace with your data
predictor = dlib.shape_predictor("/content/drive/MyDrive/Datasets/shape_predictor_68_face_landmarks.dat")  # Ensure this file is downloaded

def detect_and_align_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        # Extract the coordinates of the face landmarks
        landmarks = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)]

        # Align the face (optional: you can use affine transformation for better alignment)
        aligned_face = image[face.top():face.bottom(), face.left():face.right()]
        return aligned_face

    return None

In [31]:
from tqdm import tqdm  # Import tqdm for the progress bar

def generate_deepfake(video_path, output_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Set up the video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use a widely supported codec
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (128, 128))  # Ensure frame size matches the model's output

    # Initialize tqdm progress bar
    progress_bar = tqdm(total=total_frames, desc="Processing Frames", unit="frame")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Detect and align the face
            face = detect_and_align_face(frame)

            if face is not None:
                # Resize the face to match the model's input size
                face = cv2.resize(face, (128, 128))

                # Normalize the face image
                face = face / 255.0

                # Convert the face to a tensor and pass it through the model
                face_tensor = torch.from_numpy(face).permute(2, 0, 1).unsqueeze(0).float().to(device)
                with torch.no_grad():
                    fake_face = model(face_tensor).squeeze(0).cpu().numpy()
                    fake_face = np.transpose(fake_face, (1, 2, 0)) * 255
                    fake_face = fake_face.astype(np.uint8)

                # Write the fake face to the output video
                out.write(fake_face)
            else:
                # Write the original frame (or a placeholder) if no face is detected
                out.write(cv2.resize(frame, (128, 128)))

            # Update the progress bar
            progress_bar.update(1)
    finally:
        # Release resources
        cap.release()
        out.release()
        progress_bar.close()  # Close the progress bar

    print("\nDeepFake video generated successfully!")

In [33]:
# Generate the DeepFake video
generate_deepfake("/content/6260385433993.mp4", "deepfake_output.mp4")

Processing Frames: 100%|██████████| 309/309 [00:21<00:00, 14.54frame/s]


DeepFake video generated successfully!
